<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/slideshow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }

!pip install moviepy ipywidgets
!apt-get install -y fonts-dejavu-core

In [ ]:
#@title ##**Upload images** { display-mode: "form" }
from google.colab import files, drive
import os
import zipfile

upload_source = "Google Drive archive" #@param ["PC", "Google Drive", "Google Drive archive"] {type:"string"}

if upload_source == "PC":
    print("Please select images to upload from your PC")
    uploaded_images = files.upload()
    image_filenames = list(uploaded_images.keys())

elif upload_source == "Google Drive":
    drive.mount('/content/drive')
    folder_name = input("Please enter the name of the folder on Google Drive: ")

    folder_path = f'/content/drive/My Drive/{folder_name}'
    if not os.path.exists(folder_path):
        print(f"Folder '{folder_name}' not found on Google Drive.")
    else:
        image_filenames = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]
        if image_filenames:
            print("Images found in the folder:")
            for filename in image_filenames:
                print(filename)
        else:
            print("No images found in the specified folder.")

elif upload_source == "Google Drive archive":
    drive.mount('/content/drive')
    archive_name = input("Please enter the name of the archive on Google Drive (including extension, e.g. 'images.zip'): ")

    archive_path = f'/content/drive/My Drive/{archive_name}'
    if not os.path.exists(archive_path):
        print(f"Archive '{archive_name}' not found on Google Drive.")
    else:
        print(f"Found archive: {archive_name}")
        try:
            with zipfile.ZipFile(archive_path, 'r') as zip_ref:
                image_filenames = []
                for file_info in zip_ref.filelist:
                    if file_info.filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
                        filename = os.path.basename(file_info.filename)
                        if filename:  # Skip if filename is empty (directory)
                            zip_ref.extract(file_info, ".")  # Распаковываем в текущую директорию
                            image_filenames.append(filename)
                            print(f"Extracted: {filename}")

                if image_filenames:
                    print("\nSuccessfully extracted images")
                else:
                    print("No images found in the archive.")
        except Exception as e:
            print(f"Error processing archive: {e}")
            image_filenames = []

# Now `image_filenames` will contain the list of image files from the chosen source

In [ ]:
#@title ##**Upload audio** { display-mode: "form" }
from google.colab import files
print("Please select an audio file to upload")
uploaded_audio = files.upload()

audio_filename = list(uploaded_audio.keys())[0]

In [9]:
#@title ##**Configuration** { display-mode: "form" }
#selected_effect = "crossfade"  #@param ["crossfade", "fade", "slide_left", "slide_right", "none"] {type:"string"}
#transition_duration = 1.0  #@param {type:"number"}
with_text_option = "With text"  #@param ["With text", "Without text"] {type:"string"}
font_size = 38  #@param {type:"slider", min:10, max:100, step:1}
text_color = "#FFFFFF"  #@param {type:"string"}
import ipywidgets as widgets
from IPython.display import display

transition_effect = widgets.Dropdown(
    options=[
        ('No effect', 'none'),
        ('Crossfade', 'crossfade'),
        ('Fade in/out', 'fade'),
        ('Slide left', 'slide_left'),
        ('Slide right', 'slide_right'),
        ('Slide up', 'slide_up'),
        ('Slide down', 'slide_down')
    ],
    value='none',
    description='Transition effect:',
)

transition_duration = widgets.FloatSlider(
    value=1.0,
    min=0.0,
    max=5.0,
    step=0.1,
    description='Transition duration (sec):',
    continuous_update=False
)

display(transition_effect, transition_duration)

Dropdown(description='Transition effect:', options=(('No effect', 'none'), ('Crossfade', 'crossfade'), ('Fade …

FloatSlider(value=1.0, continuous_update=False, description='Transition duration (sec):', max=5.0)

In [ ]:
#@title ##**Run** { display-mode: "form" }
from moviepy.editor import *
from PIL import Image, ImageDraw, ImageFont

def create_image_with_text(image_path, text, font_size, text_color):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"
    font = ImageFont.truetype(font_path, font_size)

    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]

    position = ((image.width - text_width) // 2, image.height - text_height - 10)

    draw.text(position, text, fill=text_color, font=font)

    temp_image_path = "/tmp/temp_with_text.png"
    image.save(temp_image_path)
    return temp_image_path

selected_effect = transition_effect.value
transition_dur = transition_duration.value
add_text = with_text_option

audio_clip = AudioFileClip(audio_filename)
audio_duration = audio_clip.duration

num_images = len(image_filenames)
total_transition_time = (num_images - 1) * transition_dur
image_duration = (audio_duration - total_transition_time) / num_images

if image_duration <= 0:
    print("The audio duration is too short for the given number of images and transition durations")
else:
    clips = []
    for i, filename in enumerate(image_filenames):
        if add_text == 'With text':
            temp_image_path = create_image_with_text(filename, filename, font_size, text_color)
            clip = ImageClip(temp_image_path).set_duration(image_duration + transition_dur)
        else:
            clip = ImageClip(filename).set_duration(image_duration + transition_dur)

        if selected_effect == 'crossfade':
            start_time = i * image_duration
        else:
            start_time = i * (image_duration + transition_dur)

        clip = clip.set_start(start_time)

        if selected_effect == 'crossfade':
            if i > 0:
                clip = clip.crossfadein(transition_dur)
        elif selected_effect == 'fade':
            clip = clip.fx(vfx.fadein, duration=transition_dur).fx(vfx.fadeout, duration=transition_dur)
        elif selected_effect.startswith('slide_'):
            side = selected_effect.split('_')[1]
            clip = clip.fx(vfx.slide_in, duration=transition_dur, side=side)

        clips.append(clip)

    final_clip = CompositeVideoClip(clips)
    final_clip = final_clip.set_duration(audio_duration)
    final_clip = final_clip.set_audio(audio_clip)

    output_filename = "slideshow_video.mp4"
    final_clip.write_videofile(output_filename, fps=24)

In [ ]:
#@title ##**Download video** { display-mode: "form" }
from google.colab import files

files.download(output_filename)